#OpenAI API 키 설정

* Google Colab 보안 비밀에 OpenAI API 키를 "OPEN_API_KEY" 라는 이름의 환경 변수로 설정합니다.

In [1]:
import os
from google.colab import userdata

os.environ["OPEN_API_KEY"] = userdata.get("OPEN_API_KEY");              # OpenAI API 키
# os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY");  # LangSmith API 키

#LangChain 설치

In [3]:
# 핵심 패키지들
!pip install langchain>=0.3.0 langchain-openai langchain-community langchain-text-splitters

In [4]:
# 추가 유용한 패키지들
!pip install faiss-cpu tiktoken python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 84.4 MB/s eta 0:00:00


In [5]:
# LangChain v0.3+ 호환 버전
!pip install langchain-openai>=0.2.0

# RAG (Retrieval-Augumented Generation)

* 사용자의 질의에 보다 정확하고 풍부한 답변을 제공하기 위해 검색 기반 정보 보강 기법을 활용하는 생성 모델 아키텍처입니다.
* RAG의 전형적인 구성은
먼저 도메인 특화 정보나 최신 문서를 임베딩(embedding) 기법을 통해 벡터화하고, 이를 벡터 데이터베이스에 저장합니다. 사용자의 입력이 주어지면 해당 입력 역시 벡터로 변환된 후, 벡터 유사도 검색을 통해 벡터 데이터베이스에서 관련성이 높은 문서들을 검색합니다. 이후 검색된 문서들을 context로 구성하여 언어 모델의 입력에 포함시키고, 이를 기반으로 최종 응답을 생성합니다.
*  이러한 방식은 사전 학습된 모델의 한계를 보완하고, 도메인 특화 정보나 최신 데이터에 대한 접근을 가능하게 하여 보다 정밀하고 신뢰할 수 있는 응답을 생성하는 데에 유용합니다.

[RAG를 사용하지 않는 예시]

In [6]:

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 프롬프트 템플릿 객체 생성
prompt = PromptTemplate.from_template("""
문맥을 바탕으로 질문에 한 문장으로 답변해 주세요.
문맥: {context}
질문: {question}""")


model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    openai_api_key=os.environ["OPEN_API_KEY"]
)

# PromptTemplate와 모델 체인 연결
chain = prompt | model

# 예제 질문으로 체인 실행
context = """
Core benefits
LangGraph provides low-level supporting infrastructure for any long-running, stateful workflow or agent. LangGraph does not abstract prompts or architecture, and provides the following central benefits:

Durable execution: Build agents that persist through failures and can run for extended periods, automatically resuming from exactly where they left off.
Human-in-the-loop: Seamlessly incorporate human oversight by inspecting and modifying agent state at any point during execution.
Comprehensive memory: Create truly stateful agents with both short-term working memory for ongoing reasoning and long-term persistent memory across sessions.
Debugging with LangSmith: Gain deep visibility into complex agent behavior with visualization tools that trace execution paths, capture state transitions, and provide detailed runtime metrics.
Production-ready deployment: Deploy sophisticated agent systems confidently with scalable infrastructure designed to handle the unique challenges of stateful, long-running workflows.
LangGraph's ecosystem
While LangGraph can be used standalone, it also integrates seamlessly with any LangChain product, giving developers a full suite of tools for building agents. To improve your LLM application development, pair LangGraph with:

LangSmith — Helpful for agent evals and observability. Debug poor-performing LLM app runs, evaluate agent trajectories, gain visibility in production, and improve performance over time.
LangSmith Deployment — Deploy and scale agents effortlessly with a purpose-built deployment platform for long running, stateful workflows. Discover, reuse, configure, and share agents across teams — and iterate quickly with visual prototyping in LangGraph Studio.
LangChain – Provides integrations and composable components to streamline LLM application development.
Note

Looking for the JS version of LangGraph? See the JS repo and the JS docs.

Additional resources
Guides: Quick, actionable code snippets for topics such as streaming, adding memory & persistence, and design patterns (e.g. branching, subgraphs, etc.).
Reference: Detailed reference on core classes, methods, how to use the graph and checkpointing APIs, and higher-level prebuilt components.
Examples: Guided examples on getting started with LangGraph.
LangChain Forum: Connect with the community and share all of your technical questions, ideas, and feedback.
LangChain Academy: Learn the basics of LangGraph in our free, structured course.
Templates: Pre-built reference apps for common agentic workflows (e.g. ReAct agent, memory, retrieval etc.) that can be cloned and adapted.
Case studies: Hear how industry leaders use LangGraph to ship AI applications at scale.
Acknowledgements
LangGraph is inspired by Pregel and Apache Beam. The public interface draws inspiration from NetworkX. LangGraph is built by LangChain Inc, the creators of LangChain, but can be used without LangChain.
"""
question = "LangGraph의 핵심 이점은 무엇인가요?"
response = chain.invoke(
    {
      "context": context,
      "question": question
    }
)

# 응답 출력
print(response.content)

LangGraph의 핵심 이점은 내구성 있는 실행, 인간의 개입 가능성, 포괄적인 메모리, LangSmith를 통한 디버깅, 그리고 생산 준비 완료된 배포입니다.



##RAG 파이프라인

* RAG 파이프라인은 다음과 같이 5단계로 구성됩니다.
  1. 데이터 로드
  2. 텍스트 분할
  3. 인덱싱 (검색시간 단축, 검색의 정확도 향상)
  4. 검색
  5. 생성



####LangChain의 RAG 관련 컴포넌트

* Document Loader : 데이터 소스에서 문서를 읽어 들임
* Document Transformer : 문서에 어떤 변환을 가함  (텍스트 분할)
* Embedding Model : 문서를 벡터화함
* Vector Store : 벡터화한 문서 저장소
* Retriever : 사용자의 질의을 바탕으로 인덱싱된 데이터에서 관련된 문서 검색


###Document Loader

* 다양한 외부 소스(웹페이지, 문서, 데이터베이스등)에서 데이터를 LangChain의 표준 Document 형식으로 불러오는 역할을 합니다.
* 모든 문서 로더는 BaseLoader 인터페이스를 구현합니다.
* load() 메서드로 모든 문서를 한 번에 로드합니다.







####PyPDFLoader
* PDF 파일에서 간단한 텍스트 추출에 적합하며, 안정적이고 널리 사용됨
* 페이지 단위 분할 지원: 각 페이지를 별도의 Document 객체로 반환 가능.

In [ ]:

# pypdf 라이브러리 설치

!pip install -q pypdf


In [ ]:
# PDF 파일 업로드

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = 'MSA기반 서비스 분해 전략 및 서비스 IPC 개발하기.pdf'

loader = PyPDFLoader(file_path)

# 모든 문서를 한번에 로드
pages = loader.load()

len(pages)  # 페이지 단위로 Document 겍체 생성

print(pages[0])
print(pages[0].page_content)
print(pages[0].metadata)

page_content='MSA기반 서비스 분해 전략 및
서비스 IPC 개발하기
이정구
itgenius1004@gmail.com' metadata={'producer': 'Microsoft® PowerPoint® 2019', 'creator': 'Microsoft® PowerPoint® 2019', 'creationdate': '2024-07-21T08:50:21+09:00', 'title': '', 'author': 'HPE', 'keywords': '', 'moddate': '2024-07-21T08:50:21+09:00', 'source': 'MSA기반 서비스 분해 전략 및 서비스 IPC 개발하기.pdf', 'total_pages': 153, 'page': 0, 'page_label': '1'}
MSA기반 서비스 분해 전략 및
서비스 IPC 개발하기
이정구
itgenius1004@gmail.com
{'producer': 'Microsoft® PowerPoint® 2019', 'creator': 'Microsoft® PowerPoint® 2019', 'creationdate': '2024-07-21T08:50:21+09:00', 'title': '', 'author': 'HPE', 'keywords': '', 'moddate': '2024-07-21T08:50:21+09:00', 'source': 'MSA기반 서비스 분해 전략 및 서비스 IPC 개발하기.pdf', 'total_pages': 153, 'page': 0, 'page_label': '1'}


###Text Splitter
* 긴 문서를 적절한 크기의 단위(chunk)로 분할하는 과정입니다.
* LLM은 한 번에 처리할 수 있는 입력 토큰의 수에 제한이 있는데 문서를 청크로 나누면 토큰 초과 방지, 정확한 응답, 검색 최적화할 수 있습니다.

  문서를 청크로 나누는 방식
    * 고정 길이 분할 : 일정한 토큰 수나 문자 수 단위로 단순 분할
    * 의미 기반 분할 : 문단, 문장, 주제 등 의미 단위로 분할 (예: Markdown 헤더 기준, 문장 기준 등)
    * 중첩 청크 : 각 청크에 이전 청크의 일부를 겹치게 포함해 문맥을 유지


####CharacterTextSplitter
* 주어진 텍스트를 문자 단위로 분할하는 데 사용됩니다

In [ ]:
from langchain_text_splitters import  CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator = "",
    chunk_size = 100,       # 각 청크의 최대 길이, 하나의 청크에 포함될 문자수
    chunk_overlap  = 20,    # 인접한 청크 사이에 중복으로 포함될 문자의 수
    length_function = len,  # 청크의 길이를 계산하는 함수
)

texts = text_splitter.split_text(pages[5].page_content)

len(texts)

len(texts[0])  # 분할된 텍스트 조각 중에서 첫번째 청크의 길이

for i in range(0, len(texts)):
  print(f"{i}: {texts[i]}")



0: 모놀리식 아키텍처
• 모놀리식 아키텍처는 특히 초기 단계의 프로젝트나 규모가 작고 복잡성이 낮은 애플리케이션에 적합할 수
있음. 
• 애플리케이션이 성장하고 요구 사항이 복잡해지면
1: 션이 성장하고 요구 사항이 복잡해지면서 확장성이나 유지 보수성의 문제로 인해
마이크로서비스 아키텍처로 전환을 고려하는 경우가 많음.  시스템의 확장성 유지보수성 및 개발
효율성을
2: 확장성 유지보수성 및 개발
효율성을 향상시킬 수 있기 때문임.


####RecursiveCharacterTextSplitter
* 여러 구분자(["\n\n", "\n", ".", "!", "?", ",", " ", ""])를 기준으로 재귀적으로 텍스트를 나눕니다.
* 먼저 \n\n 기준으로 나눠보고, 각 조각이 너무 크면 그 다음 구분자인 \n으로, 또 너무 크면 . 으로... 이런 식으로 재귀적으로 더 잘게 나눔.
* 기본적으로 문자 길이 기준으로 분할합니다.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 20
    #separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]  # 기본값
)

texts = text_splitter.split_text(pages[5].page_content)

len(texts)

len(texts[0])

for i in range(0, len(texts)):
  print(f"{i}: {texts[i]}")

토큰 수를 기준으로 텍스트 분할 (Tokenizer 활용)
* 토큰 수를 기준으로 텍스트를 청크(chunk)로 나누기 위해 설계된 방법입니다.
* 대규모 언어 모델(LLM)을 사용할 때 모델이 처리할 수 있는 토큰 수에는 한계가 있습니다.
* LLM 모델의 Tokenizer를 기준으로 토큰 수를 계산하고, 토큰 수를 기준으로 텍스트를 적절한 크기 청크로 나누면 모델 입력 토큰 수를 조절할 수 있습니다.

In [ ]:
from langchain_text_splitters import CharacterTextSplitter


text_splitter = CharacterTextSplitter.from_tiktoken_encoder (
    chunk_size = 100,               # 각 청크의 최대 길이, 하나의 청크에 포함될 문자수
    chunk_overlap  = 20,            # 인접한 청크 사이에 중복으로 포함될 문자의 수
    encoding_name='cl100k_base'     # OpenAI 최신 모델(GPT-3.5, GPT-4 등)에서 텍스트를 토큰 단위로 변환할 때 사용하는 기본 토크나이저의 인코딩 방식
)

text = "LangChain은 LLM과 함께 사용할 수 있는 프레임워크입니다. 검색, 체인, 에이전트 등 다양한 기능을 지원합니다."

chunks = text_splitter.split_text(text)

print(chunks)

for i, chunk in enumerate(chunks):
  print(f"{i}: {chunk}")




['LangChain은 LLM과 함께 사용할 수 있는 프레임워크입니다. 검색, 체인, 에이전트 등 다양한 기능을 지원합니다.']
0: LangChain은 LLM과 함께 사용할 수 있는 프레임워크입니다. 검색, 체인, 에이전트 등 다양한 기능을 지원합니다.


In [ ]:
# 토큰으로 분할된 내용을 직접 확인하기 (tiktoken 활용)
import tiktoken

# GPT-4 또는 GPT-3.5용 tokenizer
encoding = tiktoken.get_encoding("cl100k_base")

text = "LangChain은 LLM과 함께 사용할 수 있는 프레임워크입니다. 검색, 체인, 에이전트 등 다양한 기능을 지원합니다."

# 텍스트 → 토큰 ID
token_ids = encoding.encode(text)
print("토큰 ID 목록:", token_ids)

# 토큰 ID → 실제 토큰 (문자열 조각)
tokens = [encoding.decode([t]) for t in token_ids]
print("분할된 토큰들:")
for i, token in enumerate(tokens):
    print(f"{i+1}: {repr(token)}")


# 토큰 ID 리스트 전체를 한번에 디코딩
decoded_text = encoding.decode(token_ids)
print(decoded_text)



###Embedding
* 텍스트 데이터를 숫자로 이루어진 벡터로 변환하는 과정을 말합니다
* 임베딩 벡터는 단순한 숫자 배열이 아니라,
단어들 간 의미적 관계와 문맥을 반영한 의미 공간상의 좌표라고 할 수 있습니다.

Embedding 활용 사례


1. 문서 & 정보 검색(Semantic Search)
2. 텍스트 분류 및 감성 분석
3. 챗봇/질의응답
4. 중복 문서/문장 탐지







####OpenAIEmbeddings



In [ ]:
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(openai_api_key=os.environ["OPEN_API_KEY"])

embeddings = embedding_model.embed_documents(
    [
      '안녕하세요',
      '어! 오랜만이네요',
      '이름이 어떻게 되세요?',
      '날씨가 추워요',
      'Hello!!'
    ]
)

len(embeddings)  # document 갯수

len(embeddings[0]) # 첫 번째 문서의 벡터 차원(1536), 문서 단위로 임베딩을 수행

# print(len(embeddings), len(embeddings[0]))   # (5, 1536)

print(embeddings[0][:20])



[-0.013639057986438274, -0.009446357376873493, -0.005943919066339731, -0.022944806143641472, -0.012475838884711266, 0.01820245385169983, -0.01977471634745598, 0.005880006123334169, -0.012904057279229164, 0.008788052946329117, 0.012060403823852539, -0.0032292099203914404, -0.024005763232707977, -0.014495492912828922, -0.005608375184237957, -0.007171051111072302, 0.02722698450088501, -0.01342175342142582, 0.007886878214776516, -0.017435496672987938]


In [ ]:
# embed_query 메소드는 단일 쿼리 문자열을 받아 이를 벡터 공간에 임베딩합니다.
# 주로 검색 쿼리나 질문 같은 단일 텍스트를 임베딩할 때 유용합니다.

embedded_query = embedding_model.embed_query('첫인사를 하고 이름을 물어봤나요?')

print(len(embedded_query))  # 1536차원

print(embedded_query)

1536
[0.003640108974650502, -0.024275783449411392, 0.010910888202488422, -0.04110145568847656, -0.004543057177215815, 0.021784022450447083, -0.004156079608947039, 0.02063881978392601, -0.006833462975919247, 0.007343141362071037, 0.004058548714965582, 0.0026758103631436825, 0.007947204634547234, -0.006688739638775587, -0.008595313876867294, -0.0025987294502556324, 0.013264217413961887, -0.017895366996526718, 0.005326451733708382, -0.02977527305483818, -0.0011302585480734706, -0.019921494647860527, -0.002051297342404723, 0.015076406300067902, 0.005339036229997873, 0.009778270497918129, 0.01627194881439209, -0.007192125543951988, -0.001005198690108955, -0.00360235502012074, 0.016586564481258392, 0.00944477692246437, -0.020714327692985535, -0.004511595703661442, -0.026503264904022217, 0.0013693668879568577, 0.002007251139730215, -0.005169143434613943, 0.0026176064275205135, -0.006934140343219042, 0.012376999482512474, -0.0050244200974702835, 0.011093365959823132, -0.01614610105752945, -0.0

###Vector store

*  임베딩 벡터들을 효율적으로 저장하고 검색할 수 있는 데이터베이스를 의미합니다.
* 벡터 저장소의 핵심 기능은 사용자 쿼리에 기반하여 가장 유사한 벡터(=데이터)를 빠르게 검색하는 기능을 제공합니다.
* LLM, RAG, 추천 시스템, 검색 등에서 아주 중요하게 사용됩니다.




####Chroma
* Chroma는 임베딩 벡터를 저장하기 위한 오픈소스입니다.
* 로컬에서 사용 가능한 벡터 스토어입니다.
* collection_metadata:
  * Chroma에서 벡터 저장소(컬렉션) 생성 시,벡터 간 거리 계산 방식, 인덱싱 전략 등을 설정할 수 있는 옵션
  * {'hnsw:space': 'cosine'} : 인덱싱 기법으로 HNSW 사용, 벡터 간 유사도를 Cosine Similarity 사용

Chroma 라이브러리 설치

In [ ]:
!pip install chromadb

In [ ]:
# 로컬 컴퓨터에서 파일 업로드 후 읽기

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))



Saving history.txt to history (1).txt
User uploaded file "history (1).txt" with length 2992 bytes


1. 텍스트 데이터 로드

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('history.txt')
data = loader.load()   # Document List


2. 텍스트 분할
* 텍스트를 구분자 우선순위(예: 문단 → 문장 → 쉼표 → 공백 등)를 사용해 "의미 단위"로 먼저 여러 개의 조각으로 분할하고
* 각 조각을 청크에 추가하기 전에 토큰 수를 계산하여 chunk_size를 넘지 않는 한에서 청크를 구성합니다.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250,
    chunk_overlap=50,
    encoding_name='cl100k_base' # 청크 분할 시 문자 수가 아닌 토큰 수 기준을 분할하려면 인코딩 설정 필요(GPT-4 모델에서 사용)
)

texts = text_splitter.split_text(data[0].page_content);

print(len(texts))  # 청크 갯수

print(len(texts[0]))

print(texts[0])

6
252
한국의 역사는 수천 년에 걸쳐 이어져 온 긴 여정 속에서 다양한 문화와 전통이 형성되고 발전해 왔습니다. 고조선에서 시작해 삼국 시대의 경쟁, 그리고 통일 신라와 고려를 거쳐 조선까지, 한반도는 많은 변화를 겪었습니다.

고조선은 기원전 2333년 단군왕검에 의해 세워졌다고 전해집니다. 이는 한국 역사상 최초의 국가로, 한민족의 시원이라 할 수 있습니다. 이후 기원전 1세기경에는 한반도와 만주 일대에서 여러 소국이 성장하며 삼한 시대로 접어듭니다.


3. 임베딩 모델 초기화 & Chroma 벡터 저장소 생성

In [ ]:
!pip install -U langchain-openai   # --upgrade

In [ ]:
# from langchain_community.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings

from langchain_community.vectorstores import Chroma

embedding_model = OpenAIEmbeddings(openai_api_key=os.environ["OPEN_API_KEY"])

db = Chroma.from_texts (
        texts,
        embedding_model,
        collection_name='history',
        persist_directory='./chroma_db',
        collection_metadata={'hnsw:space': 'cosine'}
)

db



#####Similarity search
# 4. 유사도 기반 검색

In [ ]:
query = '누가 한글을 창제하였나요?'

docs = db.similarity_search(query)

len(docs)

print(docs[0].page_content)

조선은 1392년 이성계에 의해 건국되어, 1910년까지 이어졌습니다. 조선 초기에는 세종대왕이 한글을 창제하여 백성들의 문해율을 높이는 등 문화적, 과학적 성취가 이루어졌습니다. 그러나 조선 후기에는 내부적으로 실학의 발전과 함께 사회적 변화가 모색되었으나, 외부로부터의 압력은 점차 커져만 갔습니다.


#####MMR 검색
* Maximum marginal revelance search : 최대 한계 관련성 검색
* 관련성과 다양성 사이의 균형을 맞추는 검색 방법
* RAG (Retrieval-Augmented Generation)이나 검색 기반 추천 시스템 등에서 사용
* 사용 예
  * 예를 들어 사용자가 어떤 질문을 했을 때, 질문과 관련이 있는 문서를 찾되 (높은 관련성)
  * 이미 선택한 문서와 너무 비슷한 문서는 제외하고 싶을 때 사용됩니다. (낮은 중복성)
  * lambda_mult(람다 값) : (0 ~ 1) 1에 가까울수록 관련성을 우선, 0에 가까울수록 다양성을 우선


**코드 예제**
1. !pip install pymupdf 명령어로 라이브러리 설치
2. PyMuPDFLoader를 사용하여 PDF 파일('323410_카카오뱅크_2023.pdf')에서 텍스트 데이터를 로드
3. RecursiveCharacterTextSplitter 텍스트 분할
4. OpenAIEmbeddings 사용하여 임베딩 모델 생성
5. 유사도 기반 검색
6. MMR 검색



In [ ]:
!pip install pymupdf

로컬 파일을 코랩에 업로드하여 읽기

In [ ]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


loader = PyMuPDFLoader("323410_카카오뱅크_2024.pdf")
pages = loader.load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250,
    chunk_overlap=50,
    encoding_name='cl100k_base'
)

documents = text_splitter.split_documents(pages)

len(documents)  # 청크 길이


In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma


embedding_model = OpenAIEmbeddings(openai_api_key=os.environ["OPEN_API_KEY"])

db1 = Chroma.from_documents (
        documents,   # 청크
        embedding_model,
        collection_name='kakao_bank_2024',
        persist_directory='./chroma_db',
        collection_metadata={'hnsw:space': 'cosine'}
)

db1



1. 유사도 검색 (Similarity search)

In [ ]:
query = '카카오뱅크의 2023년 당기순이익에 대해서 요약해서 설명주세요'

docs = db1.similarity_search(query)

print(len(docs))

print(docs[0].page_content)  #  검색 결과 중 가장 유사한 문서의 내용을 출력




4
5.3%p 개선되는 모습을 시현.  
투자의견 매수 및 목표주가 37,000원 유지 
카카오뱅크에 대해 투자의견 매수 및 목표주가 37,000원을 유지함. 1) 2023년 
교보증권 예상을 상회하는 연간 실적 시현했으며, 2) 2024년에도 대환 및 신규
대출 도입 등 대출성장을 꾸준히 보여 줄 것으로 기대됨. 3) 또한 지속적인 자
체 트래픽 유입에 따른 광고 매출 증가로 플랫폼수익 내 광고 비중이 올라가고


2. MMR 검색

In [ ]:

mmr_docs = db1.max_marginal_relevance_search(query, k=4, fetch_k=10) # 상위 10개의 관련성이 높은 문서 중에서 서로 다른 정보(다양성)를 제공하는 4개의 문서를 선택합니다.

print(len(mmr_docs))

print(mmr_docs[0].page_content)


10
5.3%p 개선되는 모습을 시현.  
투자의견 매수 및 목표주가 37,000원 유지 
카카오뱅크에 대해 투자의견 매수 및 목표주가 37,000원을 유지함. 1) 2023년 
교보증권 예상을 상회하는 연간 실적 시현했으며, 2) 2024년에도 대환 및 신규
대출 도입 등 대출성장을 꾸준히 보여 줄 것으로 기대됨. 3) 또한 지속적인 자
체 트래픽 유입에 따른 광고 매출 증가로 플랫폼수익 내 광고 비중이 올라가고


#####벡터스토어에 메타 데이터 추가

1. 각 문서에 대한 메타 데이터를 포함하는 구조를 준비합니다.
2. Chroma 벡터 스토어에 문서와 메타데이터 저장
3. 유사성 검색 수행 및 메타 데이터 활용



In [ ]:
from langchain_core.documents import Document


documents = [
    Document(
        page_content="LangChain은 대규모 언어 모델(LLM)을 사용하는 애플리케이션을 개발하기 위한 프레임워크입니다.",
        metadata={
            "title": "LangChain 소개",
            "author": "AI 개발자",
            "url": "http://example.com/langchain-intro"
        }
    ),
    Document(
        page_content="벡터 데이터베이스는 고차원 벡터를 효율적으로 저장하고 검색하는 데 특화된 데이터베이스 시스템입니다.",
        metadata={
            "title": "벡터 데이터베이스 개요",
            "author": "데이터 과학자",
            "url": "http://example.com/vector-db-overview"
        }
    )
    # 추가 문서들...
]

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# 임베딩 모델 생성
embedding_model = OpenAIEmbeddings(openai_api_key=os.environ["OPEN_API_KEY"])


# Chroma 벡터 스토어에 문서와 메타데이터 저장
# document는 임베딩되지만 metadata는 임베딩되지 않음.

db = Chroma.from_documents (
        documents,
        embedding_model,
        persist_directory='./chroma_db',
)

db



In [ ]:
query = "LangChain이란 무엇인가"

docs = db.similarity_search(query)

print(len(docs))

for doc in docs:
  print(f'내용 : {doc.page_content}')
  print(f'메타데이터 : {doc.metadata}')
  print(f'저자 : {doc.metadata["author"]}')
  print(f'제목 : {doc.metadata["title"]}')
  print('-----------------------------------------------------------------------------------------------------------------------------------')


####Retriever
* 사용자 질문(Query) 을 입력받아 관련 문서나 데이터 조각(Chunks) 을 검색(Retrieve) 해주는 역할을 합니다.


#####Vector Store Retriever
* 벡터 데이터베이스(Vector Store)에서 임베딩 기반으로 관련 문서를 검색(Retrieve)하는 객체입니다.

1. 문서 로드 및 텍스트 분할

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. 문서 로드
loader = PyMuPDFLoader("323410_카카오뱅크_2024.pdf")
data = loader.load() # Document 객체들의 리스트
print(len(data))

# 2. 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100,
    chunk_overlap=20,
    encoding_name='cl100k_base'
)


documents = text_splitter.split_documents(data)
len(documents)


3


76

2. 문서 임베딩을 벡터 스토어에 저장
* 문서를 저장할 때 벡터 인덱싱 기법(HNSW)을 적용하고,
그 인덱스를 이용해 빠르고 효율적으로 문서를 검색할 수 있도록 준비하는 것입니다.

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma


# 임베딩 모델 생성
embedding_model = OpenAIEmbeddings(openai_api_key=os.environ["OPEN_API_KEY"])

# Chroma 벡터 스토어에 문서 저장
db = Chroma.from_documents (
        documents,
        embedding_model,
        collection_name='kakao_bank_2024',
        persist_directory='./chroma_db',
        collection_metadata={'hnsw:space': 'cosine'} # HNSW (Hierarchical Navigable Small World) 인덱싱 기법 사용, 거리 함수로 Cosine Similarity 사용
)



In [ ]:
import os

print(os.listdir("./chroma_db"))
print(os.listdir("./chroma_db/4fec00fa-a8fd-4ffa-bc50-5e6f1d60f264"))



['chroma.sqlite3', '4fec00fa-a8fd-4ffa-bc50-5e6f1d60f264', 'd354b1cb-b4aa-4ea9-95b2-821d1e5445e3', 'c14859be-ac7a-4799-a746-5b1597c857ae']
['header.bin', 'data_level0.bin', 'link_lists.bin', 'length.bin']


유사도 기반 검색


In [ ]:
query = "'카카오뱅크의 2023년 당기순이익에 대해서 요약해서 설명주세요"

# retriever:  벡터 데이터베이스에서 임베딩 기반으로 관련 문서를 검색하는 VectorStoreRetriever 객체
retriever = db.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 1}
) # 검색 시 가장 유사한 문서 1개만 반환


docs = retriever.get_relevant_documents(query)

print(len(docs))
print(docs[0])



1
page_content='1.0 
-2.2 
 
 
 
2023년 당기순이익 3,549억원, YoY +34.9% 
카카오뱅크의 2023년 당기순이익은 3,549억원으로 전년대비 34.9% 증가 시' metadata={'format': 'PDF 1.6', 'keywords': '', 'author': '교보', 'creator': 'Microsoft® Word 2016', 'trapped': '', 'page': 0, 'source': '323410_카카오뱅크_2024.pdf', 'file_path': '323410_카카오뱅크_2024.pdf', 'moddate': '2024-02-13T09:37:54+09:00', 'creationDate': "D:20240213093744+09'00'", 'total_pages': 3, 'producer': 'Microsoft® Word 2016', 'title': '', 'subject': '', 'modDate': "D:20240213093754+09'00'", 'creationdate': '2024-02-13T09:37:44+09:00'}


다양성을 고려한 MMR 검색(1)
* 다양성을 고려한 MMR 검색을 사용하여 상위 5개 문서를 검색합니다

In [ ]:
query = "'카카오뱅크의 2023년 당기순이익에 대해서 요약해서 설명주세요"


# RAG 파이프라인에 통합
# 'lambda_mult': 0.5  # 관련성 vs 다양성 조절 (0~1)

retriever = db.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'fetch_k': 50,}   # 쿼리에 가장 유사한 상위 50개의 문서 중 다양성을 고려하여 5개의 문서 선택
)


mmr_docs = retriever.get_relevant_documents(query)

print(len(mmr_docs))
print(mmr_docs[0])

5
page_content='1.0 
-2.2 
 
 
 
2023년 당기순이익 3,549억원, YoY +34.9% 
카카오뱅크의 2023년 당기순이익은 3,549억원으로 전년대비 34.9% 증가 시' metadata={'file_path': '323410_카카오뱅크_2024.pdf', 'keywords': '', 'modDate': "D:20240213093754+09'00'", 'subject': '', 'creationDate': "D:20240213093744+09'00'", 'moddate': '2024-02-13T09:37:54+09:00', 'creator': 'Microsoft® Word 2016', 'producer': 'Microsoft® Word 2016', 'page': 0, 'creationdate': '2024-02-13T09:37:44+09:00', 'title': '', 'total_pages': 3, 'trapped': '', 'author': '교보', 'format': 'PDF 1.6', 'source': '323410_카카오뱅크_2024.pdf'}


다양성을 고려한 MMR 검색(2)

* 검색 결과의 관련성과 다양성을 균형있게 조정하는 방식입니다.
* lambda_mult 매개변수는 관련성과 다양성 사이의 균형을 조정합니다

In [ ]:
query = "'카카오뱅크의 2023년 당기순이익에 대해서 요약해서 설명주세요"

retriever = db.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'lambda_mult': 0.15}   # (0 ~ 1)  1에 가까울수록 관련성을 우선, 0에 가까울수록 다양성을 우선한다.
)

mmr_docs = retriever.get_relevant_documents(query)

print(len(mmr_docs))
print(mmr_docs[0])

5
page_content='1.0 
-2.2 
 
 
 
2023년 당기순이익 3,549억원, YoY +34.9% 
카카오뱅크의 2023년 당기순이익은 3,549억원으로 전년대비 34.9% 증가 시' metadata={'moddate': '2024-02-13T09:37:54+09:00', 'source': '323410_카카오뱅크_2024.pdf', 'total_pages': 3, 'creationdate': '2024-02-13T09:37:44+09:00', 'keywords': '', 'creator': 'Microsoft® Word 2016', 'file_path': '323410_카카오뱅크_2024.pdf', 'format': 'PDF 1.6', 'trapped': '', 'author': '교보', 'page': 0, 'title': '', 'modDate': "D:20240213093754+09'00'", 'creationDate': "D:20240213093744+09'00'", 'producer': 'Microsoft® Word 2016', 'subject': ''}


유사도 점수 임계값 기반 검색
* score_threshold 유사도 점수 이상인 문서만을 대상으로 추출합니다
* 유사도가 높은 문서만 필터링하고 싶을 때 유용합니다.


In [ ]:
query = "'카카오뱅크의 2023년 당기순이익에 대해서 요약해서 설명주세요"

retriever = db.as_retriever(
    search_type='similarity_score_threshold',
    search_kwargs={'score_threshold': 0.3}  # Cosine Similarity (-1 ~ 1) : 1에 가깔울수록 매우 유사
)


docs = retriever.get_relevant_documents(query)

print(len(docs))
print(docs[0])


4
page_content='1.0 
-2.2 
 
 
 
2023년 당기순이익 3,549억원, YoY +34.9% 
카카오뱅크의 2023년 당기순이익은 3,549억원으로 전년대비 34.9% 증가 시' metadata={'creationDate': "D:20240213093744+09'00'", 'producer': 'Microsoft® Word 2016', 'file_path': '323410_카카오뱅크_2024.pdf', 'creator': 'Microsoft® Word 2016', 'source': '323410_카카오뱅크_2024.pdf', 'trapped': '', 'format': 'PDF 1.6', 'subject': '', 'title': '', 'keywords': '', 'total_pages': 3, 'creationdate': '2024-02-13T09:37:44+09:00', 'page': 0, 'author': '교보', 'moddate': '2024-02-13T09:37:54+09:00', 'modDate': "D:20240213093754+09'00'"}


#####Multi Query Retriever
* https://python.langchain.com/docs/how_to/MultiQueryRetriever/
* RAG 에서 하나의 질문을 여러 방식으로 재표현해서 더 풍부하고 관련성 높은 검색 결과를 얻기 위한 방법입니다.
* 하나의 쿼리를 LLM을 이용해 여러 개의 다양한 표현으로 확장한 후,
각 표현을 벡터 검색하여 더 폭넓고 정밀한 문서 검색을 가능하게 합니다.

[작동 방식]
1. 사용자의 질문: "카카오뱅크의 2023년 순이익은?"
2. MultiQueryRetriever는 내부적으로 LLM을 이용해 이 질문을 다양한 표현으로 재작성:
   * "카카오뱅크 2023년 실적"
   * "카카오뱅크의 작년 수익"
   * "2023년 카카오뱅크가 벌어들인 돈은?"
3. 각 쿼리에 대해 개별적으로 벡터 검색 수행
4. 검색된 결과들을 합쳐서 중복 제거 후, LLM에게 전달

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


# 1. 문서 로드
loader = PyMuPDFLoader("323410_카카오뱅크_2024.pdf")
documents = loader.load() # Document 객체들의 리스트
print(len(documents))


# 2. 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100,
    chunk_overlap=20,
    encoding_name='cl100k_base'
)


# 임베딩 모델 생성
embedding_model = OpenAIEmbeddings(openai_api_key=os.environ["OPEN_API_KEY"])


# Chroma 벡터 스토어에 문서 저장
db = Chroma.from_documents (
        documents,
        embedding_model,
        collection_name='kakao_bank_2024',
        persist_directory='./chroma_db',
        collection_metadata={'hnsw:space': 'cosine'} # HNSW (Hierarchical Navigable Small World) 인덱싱 기법 사용, 거리 함수로 Cosine Similarity 사용
)

print(db)



3


In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chat_models import ChatOpenAI

question = "카카오 뱅크의 최근 영업 실적을 알려주세요"

model = ChatOpenAI(
    model="gpt-4o-mini",  # 모델
    temperature=0,        # 0 ~ 2 사이에서 선택, 0.8과 같은 높은 값은 무작위하게 출력, 0.2와 같은 낮은 값은 결정론적 출력을 생성합니다.
    max_tokens=500,
    openai_api_key=os.environ["OPEN_API_KEY"])


retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(),
    llm=model,
)


In [ ]:
!pip show langchain

In [ ]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


# Prompt
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

# Model
llm = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0,
)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# Chain
chain = (
    {'context': retriever_from_llm | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Run
response = chain.invoke('카카오뱅크의 최근 영업실적을 요약해서 알려주세요.')
response







